In [1]:
pip install torch transformers pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install shutil

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

2024-03-04 07:01:42.348944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 07:01:42.349042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 07:01:42.493836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
df=pd.read_csv('/kaggle/input/rajkishor/MajorFinal.csv')

In [6]:
df['text']= df['Question'] + " " + df['Answer']
df['text'].to_csv('/kaggle/working/MajorFinaltrain1.txt', index=False, header=False)

In [7]:
df

,Topic,Question,Answer,text
0,Dharan,What is there at Dharan to be considered as a ...,"Dharan, located in the Koshi Zone of eastern N...",What is there at Dharan to be considered as a ...
1,Dharan,what should i try to visit in dharan?,dharan in the koshi zone of eastern nepal is a...,what should i try to visit in dharan? dharan i...
2,Dharan,What is the recommendation for the tourist fir...,"For first-time visitors to Dharan, it is recom...",What is the recommendation for the tourist fir...
3,Dharan,tell me the best way to start the dharan tour?,for first-time visitors to dharan it is recomm...,tell me the best way to start the dharan tour?...
4,Dharan,What is the Dharan culture and traditions?,The Dharan Culture and traditions are deeply r...,What is the Dharan culture and traditions? The...
...,...,...,...,...
50923,Trekking in Nepal,What challenges do trekkers face with transpor...,Transportation to the starting points of treks...,What challenges do trekkers face with transpor...
50924,Trekking in Nepal,How many distinct and diverse vegetation zones...,Nepal has six distinct and diverse vegetation ...,How many distinct and diverse vegetation zones...
50925,Trekking in Nepal,Which rare species might be encountered in the...,"In the higher altitudes, one might encounter r...",Which rare species might be encountered in the...
50926,Trekking in Nepal,What accommodations are available on popular t...,Lodge accommodation is available on popular tr...,What accommodations are available on popular t...


In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('/kaggle/input/qa_model/transformers/v1/4/neha_kaggle_trained_gpt2/question_answermodel-t5')

In [10]:
#layer freezing teachnique
named_params = list(model.named_parameters())
total_params = sum(p.numel() for p in model.parameters())


# Define the layers to freeze
layers_to_freeze = range(120)

#Freeze the specified layer
for index, (name, param) in enumerate(named_params):
    if index in layers_to_freeze:
        param.requires_grad = False

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# Print the freezing status and count of trainable parameters
for name, param in model.named_parameters():
    print(f"{name}, Requires Grad: {param.requires_grad}")

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")

transformer.wte.weight, Requires Grad: False
transformer.wpe.weight, Requires Grad: False
transformer.h.0.ln_1.weight, Requires Grad: False
transformer.h.0.ln_1.bias, Requires Grad: False
transformer.h.0.attn.c_attn.weight, Requires Grad: False
transformer.h.0.attn.c_attn.bias, Requires Grad: False
transformer.h.0.attn.c_proj.weight, Requires Grad: False
transformer.h.0.attn.c_proj.bias, Requires Grad: False
transformer.h.0.ln_2.weight, Requires Grad: False
transformer.h.0.ln_2.bias, Requires Grad: False
transformer.h.0.mlp.c_fc.weight, Requires Grad: False
transformer.h.0.mlp.c_fc.bias, Requires Grad: False
transformer.h.0.mlp.c_proj.weight, Requires Grad: False
transformer.h.0.mlp.c_proj.bias, Requires Grad: False
transformer.h.1.ln_1.weight, Requires Grad: False
transformer.h.1.ln_1.bias, Requires Grad: False
transformer.h.1.attn.c_attn.weight, Requires Grad: False
transformer.h.1.attn.c_attn.bias, Requires Grad: False
transformer.h.1.attn.c_proj.weight, Requires Grad: False
transfo

In [11]:
model = model.to("cuda")

In [12]:
# Tokenize the text
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/kaggle/working/MajorFinaltrain1.txt',
    block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/question_answermodel-gpt2',
    overwrite_output_dir=True,
    num_train_epochs=40,
    logging_dir='/kaggle/working/question_answermodel-gpt2',
    save_strategy="epoch",
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start fine-tuning
trainer.train()
#trainer.train(resume_from_checkpoint=True)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,0.283300
1000,0.284100
1500,0.287100
2000,0.280100
2500,0.282000
3000,0.283400
3500,0.276700
4000,0.278600
4500,0.280400
5000,0.276400


TrainOutput(global_step=61320, training_loss=0.25698318493887246, metrics={'train_runtime': 17979.4906, 'train_samples_per_second': 109.133, 'train_steps_per_second': 3.411, 'total_flos': 1.2817419337728e+17, 'train_loss': 0.25698318493887246, 'epoch': 40.0})

In [14]:
trainer.save_model()

In [15]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)
zip_dir()

/kaggle/working/directory.zip

In [16]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [11]:
#from transformers import GPT2Model

In [29]:
model_path = '/kaggle/working/question_answermodel-gpt2'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained(model_path)

def generate_answer(question, max_length=250):

    # Encode the question to tensor of integers using the tokenizer
    input_ids = tokenizer.encode(question, return_tensors='pt')

    # Generate a sequence of tokens in response to the input question
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

    # Decode the generated ids to a string
    answer = tokenizer.decode(output[0], skip_special_tokens=True,temperature=0,top_p=0.85,top_k=20,repetition_penalty=1.5,do_sample=False,early_stopping=True,no_repeat_ngram_size=4)
    return answer

# Example question
question = "What is there at Dharan to be considered as a tourist destination?"
answer = generate_answer(question)
print(answer)

What is there at Dharan to be considered as a tourist destination? Dharan, located in the Koshi Zone of eastern Nepal, is a city with a rich cultural heritage and natural beauty that makes it an attractive tourist destination. While it may not be as well-known as other popular tourist spots in Nepal like Kathmandu or Pokhara, Dharan offers a unique experience for visitors.
One of the main attractions in Dharan is the Buda Subba Temple, dedicated to Lord Buddha. This temple is perched on a hilltop and offers panoramic views of the city and the surrounding mountains. The serene ambiance of the temple and the beautiful views of the lush green hills make it a must-visit place in Dharan. The city is also known for its cultural richness, with various local festivals and events that showcase the traditions of the region.
The cultural diversity in Dharan is also a delight, with the citys showing a blend of different ethnic groups, including the Limbu, Rai, Gurung, and Newar communities. The pr